In [1]:
!pip install PyMuPDF transformers sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 27.0 MB/s eta 0:00:00


In [2]:
from google.colab import files
import fitz  # PyMuPDF
import re

# Upload file
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Extract text
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

raw_text = extract_text_from_pdf(pdf_path)

# Clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'References.*', '', text, flags=re.IGNORECASE)
    return text.strip()

cleaned_text = clean_text(raw_text)

print("✅ Sample of Cleaned Text:\n")
print(cleaned_text[:1000])  # Preview first 1000 chars


Saving module 7_Recent_Development_of_Drone_Technology_Software_Engineering_A_Systematic_Survey.pdf to module 7_Recent_Development_of_Drone_Technology_Software_Engineering_A_Systematic_Survey.pdf
✅ Sample of Cleaned Text:

Received 28 July 2024, accepted 2 September 2024, date of publication 5 September 2024, date of current version 20 September 2024. Digital Object Identifier 10.1109/ACCESS.2024.3454546 Recent Development of Drone Technology Software Engineering: A Systematic Survey NGOC-BAO-VAN LE 1,2,*, HONG-DANH THAI 1,2,*, CHANG-WON YOON1,2,3,4, (Member, IEEE), AND JUN-HO HUH 2,5, (Member, IEEE) 1Department of Data Informatics, National Korea Maritime and Ocean University, Yeongdo-gu, Busan 49112, Republic of Korea 2Department of Interdisciplinary Major of Ocean Renewable Energy Engineering, National Korea Maritime and Ocean University, Yeongdo-gu, Busan 49112, Republic of Korea 3Department of Artificial Intelligence Application, Dongseo University, Busan 47011, Republic of Korea 

In [3]:
def split_text(text, max_chars=800):
    return [text[i:i + max_chars] for i in range(0, len(text), max_chars)]

chunks = split_text(cleaned_text)
print(f"🔹 Text split into {len(chunks)} chunks.")


🔹 Text split into 122 chunks.


In [4]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Use improved model
model_name = "mrm8488/t5-base-finetuned-question-generation-ap"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_questions(text_chunk):
    input_text = f"generate questions: {text_chunk}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    outputs = model.generate(input_ids, max_length=128, num_return_sequences=2, do_sample=True)
    return [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [5]:
all_questions = []
for chunk in chunks[:3]:  # Use only first few chunks to test
    qs = generate_questions(chunk)
    all_questions.extend(qs)

print("✅ Sample Questions Generated:\n")
for q in all_questions:
    print("•", q)


✅ Sample Questions Generated:

• question: How was this research criticized
• question: In which article were recent developments of Drone Technology Software Engineering?
• question: Which university in Korea provides support for research on drones?
• question: One location where the Department of Data Science is located?
• question: What types of questions are generated in our research?
• question: What will the literature review methodology also generate?


In [6]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.3 MB/s eta 0:00:00


In [7]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import textwrap

def save_questions_to_pdf(questions, filename="quiz_questions.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    y = height - 50
    c.setFont("Helvetica", 12)

    for i, q in enumerate(questions, start=1):
        lines = textwrap.wrap(f"{i}. {q}", 100)
        for line in lines:
            c.drawString(50, y, line)
            y -= 20
            if y < 50:
                c.showPage()
                c.setFont("Helvetica", 12)
                y = height - 50

    c.save()
    print("✅ PDF created:", filename)

# Save the questions
save_questions_to_pdf(all_questions)


✅ PDF created: quiz_questions.pdf


In [8]:
from google.colab import files
files.download("quiz_questions.pdf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
!apt-get install poppler-utils
!pip install pytesseract pdf2image transformers sentencepiece reportlab


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (747 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [10]:
!apt install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [11]:
from pdf2image import convert_from_path
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Convert PDF to images
images = convert_from_path(pdf_path)
print(f"✅ Extracted {len(images)} page images from PDF")


Saving Doc1.pdf to Doc1.pdf
✅ Extracted 1 page images from PDF


In [12]:
import pytesseract

ocr_text = ""
for img in images:
    text = pytesseract.image_to_string(img)
    ocr_text += text + "\n"

print("✅ Sample OCR Text:")
print(ocr_text[:1000])


✅ Sample OCR Text:
map. Then, a model of the LIDAR measurement error due
to the change of the scanning surface during attitude control
is built and compensated based on supporting information
from MEMS INS and the indoor environment hypothesis.
The relative position parameters calculated from the delay
matching algorithm of the environmental features and the
difference of LIDAR measurements at adjacent times are used
to estimate the error of the MEMS INS and the sensor. MEMS
using Kalman filter. The experimental results demonstrated
the potential of using the LIDAR/MEMS IMU integrated
positioning system in indoor environments.

Second, path planning algorithms enable UAVs to deter-
mine the most efficient route for moving from the initial
location to the desired destination while avoiding obsta-
cles and minimizing expenses. Commonly used algorithms
include the A-star (A*) algorithm, Dijkstra algorithm, and
Rapidly Exploring (RRT).

The paper [69] presents the issue of path planning fo

In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "iarfmoose/t5-base-question-generator"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_qas(text_chunk):
    input_text = f"generate questions: {text_chunk}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    outputs = model.generate(input_ids, max_length=128, num_return_sequences=1, do_sample=True)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # "Answer" is approximate — take the most relevant sentence from chunk
    answer = text_chunk.strip().split(". ")[0] + "."
    return question.strip(), answer.strip()

# Clean and chunk text
import re
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

cleaned_text = clean_text(ocr_text)

def split_text(text, max_chars=600):
    return [text[i:i + max_chars] for i in range(0, len(text), max_chars)]

chunks = split_text(cleaned_text)

# Generate QAs
qas = []
for chunk in chunks[:3]:  # limit for test
    q, a = generate_qas(chunk)
    qas.append((q, a))

print("✅ Sample Q&As:")
for i, (q, a) in enumerate(qas, 1):
    print(f"{i}. Q: {q}\n   A: {a}\n")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

✅ Sample Q&As:
1. Q: How to generate the questions in the MEMS/lidar-mms module? The results indicated the potential of using MEMS INS in asymmetrical imaging.
   A: map.

2. Q: Do you have any experience with path planning?
   A: r environments.

3. Q: Why awback of prolonged duration in achieving the intended objective? . The paper uses the A-star algorithm but needs to rasterize the maze to map the path.
   A: awback of prolonged duration in achieving the intended objective and inadequate tracking stability.



In [ ]:
files.download("quiz_with_answers.pdf")
